In [4]:
import requests
from bs4 import BeautifulSoup
from sympy import per
import yfinance as yf
import pandas as pd
import numpy as np

stocks = ['XENE', 'NEX', 'OXY', 'KR', 'XOM', 'WMT', 'JNJ', 'KO', 'GOOGL', 'MA', 'AAPL', 'DDOG', 'PBF', 'AMZN', 'TSLA',
         'PEP', 'MSFT', 'MCD', 'NVDA', 'GS', 'JPM', 'SQ', 'PLTR', 'ROKU', 'TDOC']

PFA, v, score, stockslist, industry, market_cap, short_float, price, beta, dividend = [], [], [], [], [], [], [], [], [], []

for stock in stocks:
    ticker = yf.Ticker(stock)
    
    #Current Price
    try:
        current_price = ticker.info['currentPrice']
    except:
        current_price = 0
        
    #52-Week High
    try:
        ATH = ticker.info['fiftyTwoWeekHigh']
    except:
        ATH = 1
    percent_from_52 = round((1 - current_price/ATH)*100, 2)
    
    if percent_from_52 == 100:
        PFA.append(0)
    else:
         PFA.append(str(percent_from_52) + '%')
    
    #Volatility
    url = f'https://www.alphaquery.com/stock/{stock}/volatility-option-statistics/180-day/historical-volatility'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    
    try:
        volatility = soup.findAll('div', class_ = "indicator-figure-inner")[0]
        vol = float(volatility.text)*100
        vol = round(vol, 2)
        if vol == 0:
            v.append("No data")
        else:
            v.append(str(round(vol, 1)) + '%')
    except:
        vol = "No Data"
        v.append(vol)
    
    #Market Cap
    try:
        mkcp = ticker.info['marketCap']
        market_cap.append(round(mkcp/1000000000))
    except:
        mkcp = 0
        market_cap.append("No data")
 
    #Industry
    try:
        industry.append(ticker.info['sector'])
    except:
        industry.append("No data")
    
    #Short Float
    try:
        shorts = ticker.info['shortPercentOfFloat']
        shorts = round(float(shorts*100), 2)
        shortss = str(shorts) + '%'
        short_float.append(shortss)
    except:
        shorts = 0
        short_float.append("No data")
    
    #Strength Score
    if vol == 0:
        score.append("N/A")
    else:
        strength_score = ((0-percent_from_52 + (vol/1.5))/2 + (shorts/5))/2
        score.append(round(strength_score, 1))
        
    stockslist.append(stock)    
    price.append('$' + str(round(current_price, 2)))
        
data = {'Stock': stockslist,
        'Industry': industry,
        'Current Price': price,
        'Market Cap (B)': market_cap,
        '% Float Short': short_float,
        '% From 52w High': PFA,
        'Volatility': v,
        'Strength Score': score}

df = pd.DataFrame(data)
df = df.sort_values(by='Strength Score', ascending=False)
df.reset_index(drop = True, inplace=True)
df.index = np.arange(1, len(df)+1)

df

#Date: September 16, 2022

,Stock,Industry,Current Price,Market Cap (B),% Float Short,% From 52w High,Volatility,Strength Score
1,XENE,Healthcare,$37.31,2,3.84%,9.86%,62.2%,8.3
2,OXY,Energy,$64.41,60,13.8%,16.49%,53.2%,6.1
3,TSLA,Consumer Cyclical,$303.35,951,2.37%,26.81%,64.7%,4.3
4,PBF,Energy,$28.33,3,16.74%,35.79%,69.6%,4.3
5,NEX,Energy,$8.45,2,3.27%,32.4%,72.0%,4.2
6,XOM,Energy,$93.21,388,0.96%,11.71%,36.6%,3.3
7,MCD,Consumer Cyclical,$254.91,188,0.77%,5.99%,19.7%,1.9
8,AAPL,Technology,$150.7,2422,0.7%,17.62%,35.5%,1.6
9,PEP,Consumer Defensive,$166.97,230,0.71%,7.79%,20.5%,1.5
10,WMT,Consumer Defensive,$133.19,362,1.06%,17.15%,32.1%,1.2
